In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.metrics import silhouette_score
import statsmodels.api
import statsmodels.formula.api as smf
from scipy import stats
from scipy.spatial.distance import cdist
#from yellowbrick.cluster import SilhouetteVisualizer


from sklearn.pipeline import make_pipeline

seed = 100

df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Pakistan Largest Ecommerce Dataset.csv");

Mounted at /content/drive


In [2]:
df.dropna(axis=0,how='all',inplace=True)
df.dropna(axis=1,how='all',inplace=True)
df=df.drop(["sales_commission_code"],axis=1)
df=df.dropna(axis=0,how="any")

df['status_new'] = df['status']
df['status_new'] = df['status_new'].replace(['order_refunded', 'refund'],'refunded')
df['status_new'] = df['status_new'].replace(['complete', 'closed'],'completed')
df['status_new'] = df['status_new'].replace(['paid','received','cod',"exchange"],'processing')
df['status_new'] = df['status_new'].replace(['holded','pending_paypal','payment_review'],'pending')

df['grand_item']=df["price"]*df["qty_ordered"] - df["discount_amount"]
test=df.groupby(["increment_id"]).agg({"grand_item":"sum","grand_total":"mean","item_id":"count"})

df=df[df["price"]>0]
df=df[df["grand_total"]>0]
df=df[df["discount_amount"]>=0]

test=df.groupby(["increment_id"]).agg({"grand_item":"sum","grand_total":"mean","item_id":"count"})

test2=test[test["grand_item"] - test["grand_total"]==0]

dele=[]
for incid in df["increment_id"]:
  dele.append(incid in test2.index)

dft=df.loc[dele]
df=dft

df['discount_rate'] = (df['discount_amount']/(df["price"]*df["qty_ordered"])* 100).round(2)

df.category_name_1.fillna('\\N',inplace=True)
df.category_name_1.replace('\\N',"Others",inplace=True)


df["BI Status"]=df["BI Status"].replace(to_replace=['#REF!'],value=["Net"])
df['BIstatus_num'] = df['BI Status']
df['BIstatus_num'] = df['BIstatus_num'].replace(['Net'], 1)
df['BIstatus_num'] = df['BIstatus_num'].replace(['Valid'], 0)
df['BIstatus_num'] = df['BIstatus_num'].replace(['Gross'], -1)

df['status_new'] = df['status']
df['status_new'] = df['status_new'].replace(['order_refunded', 'refund'],'refunded')
df['status_new'] = df['status_new'].replace(['complete', 'closed'],'completed')
df['status_new'] = df['status_new'].replace(['paid','received','cod',"exchange"],'processing')
df['status_new'] = df['status_new'].replace(['holded','pending_paypal','payment_review'],'pending')

df['payment_method'].replace(['Easypay_MA','easypay_voucher'],'Easypay',inplace=True)
df['payment_method'].replace(['jazzwallet','jazzvoucher'],'Jazz',inplace=True)
df['payment_method'].replace(['customercredit','ublcreditcard'],'Credit',inplace=True)
df['payment_method'].replace(['internetbanking'],'bankalfalah',inplace=True)

df['Working Date2'] = pd.to_datetime(df['Working Date'])
base = df['Working Date2'].max()
df['Date'] = df['Working Date2'].apply(lambda x: x.date())
df['Customer_Since_Date'] = df.groupby('Customer ID')['Date'].transform(min)
df['Lifetime'] = (base.date() - df['Customer_Since_Date']).astype('timedelta64[D]') + 1


In [3]:
#for RFM models
df=df[(df["status_new"]=="completed")|(df["status_new"]=="processing")]

T1=df.groupby(["Customer ID"]).agg({"increment_id":"count"})
m1=T1["increment_id"].mean()
s1=T1["increment_id"].std()
A1=T1[T1["increment_id"]<=m1+3*s1]
a1=A1.index
lis=[]
for id in df["Customer ID"]:
  lis.append(id in a1)

df1=df.iloc[lis]


# price, qty_ordered, discount_rate (for Monetary)
zpmax=df["price"].mean()+3*df["price"].std()
zpmin=df["price"].mean()-3*df["price"].std()
data=df1[(df1["price"]>=zpmin) & (df1["price"]<=zpmax)]

zqmax=data["qty_ordered"].mean()+3*data["qty_ordered"].std()
zqmin=data["qty_ordered"].mean()-3*data["qty_ordered"].std()
data=data[(data["qty_ordered"]>=zqmin) & (data["qty_ordered"]<=zqmax)]

zdmax=data["discount_rate"].mean()+3*data["discount_rate"].std()
zdmin=data["discount_rate"].mean()-3*data["discount_rate"].std()
data=data[(data["discount_rate"]>=zdmin) & (data["discount_rate"]<=zdmax)]



In [4]:
# Calculate RFM-scores:
# Absolute table
# Recency: Days since last purchase

data['Working Date'] = pd.to_datetime(data['Working Date'])
base = data['Working Date'].max()
data['Date'] = data['Working Date'].apply(lambda x: x.date())
data['Most_Recent_Purchase'] = data.groupby('Customer ID')['Date'].transform(max)
data['Recency'] = (base.date() - data['Most_Recent_Purchase']).astype('timedelta64[D]')

# Frequency: Total number of purchases
data['Frequency'] = data.groupby('Customer ID')['increment_id'].transform('nunique')

# Monetary: Total amount spend
data['Monetary'] = data.groupby('Customer ID')['grand_item'].transform('sum')

#RFM table
RFM_abs= data[['Customer ID', 'Recency', 'Frequency', 'Monetary']].drop_duplicates()
RFM_plot=RFM_abs.set_index("Customer ID")

In [5]:
#RFM table relative
data['Working Date3'] = pd.to_datetime(data['Working Date'])
base = data['Working Date3'].max()
data['Date'] = data['Working Date3'].apply(lambda x: x.date())
data['Customer_Since_Date'] = data.groupby('Customer ID')['Date'].transform(min)
data['Lifetime'] = (base.date() - data['Customer_Since_Date']).astype('timedelta64[D]') + 1

data['Rel. Frequency'] = data['Frequency'] / (data['Lifetime']) * 100
RFM_rel= data[['Customer ID', 'Recency', 'Rel. Frequency', 'Monetary']].drop_duplicates()
RFM_plot2=RFM_rel.set_index("Customer ID")

In [6]:
#RFM table frequency score
RFM_score_abs=RFM_abs.copy()
RFM_score_abs["Recency"]=pd.qcut(RFM_score_abs["Recency"],5,labels=[5,4,3,2,1])
RFM_abs["Frequency"][RFM_abs["Frequency"]!=1].value_counts(normalize=True)
RFM_score_abs["Frequency"]=pd.cut(RFM_score_abs["Frequency"],bins=[0,1,2,3,6,60],labels=[1,2,3,4,5])
RFM_score_abs["Monetary"]=pd.qcut(RFM_score_abs["Monetary"],5,labels=[1,2,3,4,5])
RFM_score_abs_plot= RFM_score_abs.set_index("Customer ID")

In [7]:
#RFM table score, rel. frequency
RFM_score_rel=RFM_rel.copy()
RFM_score_rel["Recency"]=pd.qcut(RFM_score_rel["Recency"],5,labels=[5,4,3,2,1])
RFM_score_rel["Rel. Frequency"]=pd.qcut(RFM_score_rel["Rel. Frequency"],5,labels=[1,2,3,4,5])
RFM_score_rel["Monetary"]=pd.qcut(RFM_score_rel["Monetary"],5,labels=[1,2,3,4,5])
RFM_score_rel_plot= RFM_score_rel.set_index("Customer ID")